<a href="https://colab.research.google.com/github/nicowilliamxvii/TQHDC_CS441/blob/main/homework_pandas_manipulation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup environment and load data

Dataset: https://s3.hothienlac.com/yomitoon/sales_data.csv

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
import numpy as np

In [4]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/cacfilecsvbaitaptqhdl/sales_data.csv')
head = df.head()
print(head)

   order_id  order_date customer_id customer_name         city       product  \
0      1001  2024-01-02        C001         Alice     New York        Laptop   
1      1002  2024-01-02        C002           Bob  Los Angeles    Headphones   
2      1003  2024-01-03        C003       Charlie     New York  Office Chair   
3      1004  2024-01-03        C001         Alice     New York         Mouse   
4      1005  2024-01-04        C004         Diana      Chicago          Desk   

      category  quantity  unit_price payment_method  
0  Electronics         1        1200    Credit Card  
1  Electronics         2         150         PayPal  
2    Furniture         1         350    Credit Card  
3  Electronics         3          25     Debit Card  
4    Furniture         1         500  Bank Transfer  


# 🟡 LEVEL 4 — Analytical Aggregation (Score 4–6)

## **Q8. Average order value (AOV) per customer**

### Task

For each customer, compute:

* total spending
* number of orders
* **average order value**

### 💡 Hint

Use:

* `.groupby()`
* `.agg()`
* basic arithmetic between aggregated columns

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)
* [https://pandas.pydata.org/docs/user_guide/groupby.html](https://pandas.pydata.org/docs/user_guide/groupby.html)

### 🧠 Explanation

This teaches:

* multi-metric aggregation
* **ratio metrics** (very common in dashboards)
* separating *raw data* from *business KPIs*

In [5]:
df['total_amount'] = df['quantity'] * df['unit_price']
aov = (
    df.groupby('customer_id')
    .agg(
        total_spending=('total_amount', 'sum'),
        number_of_orders=('order_id', 'nunique')
    )
)
aov['average_order_value'] = aov['total_spending'] / aov['number_of_orders']

In [6]:
aov.head()

,total_spending,number_of_orders,average_order_value
customer_id,,,
C001,1355,3,451.666667
C002,900,2,450.000000
C003,440,2,220.000000
C004,500,1,500.000000
C005,1100,1,1100.000000


## **Q9. Revenue contribution by category (%)**

### Task

Calculate:

* total revenue per product category
* percentage contribution of each category to total revenue

### 💡 Hint

Use:

* `.groupby()`
* `.sum()`
* `.assign()`
* division by a **global scalar**

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.div.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.div.html)

### 🧠 Explanation

You learn:

* **normalization**
* how to compare groups on the same scale
* how to prepare data for **pie charts / stacked bars**

In [7]:
df.columns
df["total_amount"] = df["unit_price"] * df["quantity"]


In [8]:
revenue_by_category = (
    df.groupby("category", as_index=False)
      .agg(total_revenue=("total_amount", "sum"))
)
revenue_by_category

,category,total_revenue
0,Electronics,3955
1,Furniture,940


In [13]:

total_revenue_all = revenue_by_category["total_revenue"].sum()

revenue_by_category = revenue_by_category.assign(
    revenue_pct = revenue_by_category["total_revenue"] / total_revenue_all * 100
)
total_revenue_all

np.int64(4895)

# 🔵 LEVEL 5 — Distribution, Ranking & Segmentation (Score 7–8)

## **Q10. Identify top 20% customers by revenue (Pareto analysis)**

### Task

Determine:

* which customers belong to the **top 20%** by total spending

### 💡 Hint

Use:

* `.sort_values()`
* `.cumsum()`
* `.quantile()`

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.quantile.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.quantile.html)

### 🧠 Explanation

This is a classic **80/20 rule** problem:

* who really drives revenue?
* foundational for **customer segmentation**

In [14]:
customer_revenue = (
    df.groupby("customer_id", as_index=False)
      .agg(total_spent=("total_amount", "sum"))
)


In [15]:
customer_revenue = customer_revenue.sort_values(
    by="total_spent", ascending=False
)


In [ ]:
customer_revenue["cumulative_revenue_pct"] = (
    customer_revenue["total_spent"].cumsum()
    / customer_revenue["total_spent"].sum()
)
top_20_customers = customer_revenue[
    customer_revenue["cumulative_revenue_pct"] <= 0.8
]
top_20_customers


In [ ]:
#use quantile
threshold = customer_revenue["total_spent"].quantile(0.8)

top_20_customers = customer_revenue[
    customer_revenue["total_spent"] >= threshold
]
top_20_customers

## **Q11. Price distribution analysis per category**

### Task

For each product category, compute:

* mean unit price
* median unit price
* standard deviation

### 💡 Hint

Use:

* `.groupby()`
* `.agg(mean=..., median=..., std=...)`

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html)

### 🧠 Explanation

This builds intuition for:

* **distribution shape**
* why **median ≠ mean**
* choosing the right chart (boxplot vs bar)

In [22]:
price_stats_by_category = (
    df.groupby("category")
      .agg(
          mean_price=("unit_price", "mean"),
          median_price=("unit_price", "median"),
          std_price=("unit_price", "std")
      )
      .reset_index()
)

In [ ]:
#price_stats_by_category


# 🔴 LEVEL 6 — Time Series, Growth & Insight (Score 9–10)

## **Q12. Day-over-day revenue growth (%)**

### Task

Compute:

* daily revenue
* **percentage change compared to previous day**

### 💡 Hint

Use:

* `.groupby()`
* `.pct_change()`

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.pct_change.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.pct_change.html)

### 🧠 Explanation

This teaches:

* growth vs absolute value
* preparing data for **line charts**
* understanding volatility

In [25]:
#df["order_date"] = pd.to_datetime(df["order_date"])
daily_revenue = (
    df.groupby("order_date", as_index=False)
      .agg(daily_revenue=("total_amount", "sum"))
      .sort_values("order_date")
)


In [ ]:
daily_revenue["daily_growth_pct"] = (
    daily_revenue["daily_revenue"].pct_change() * 100
)
#daily_revenue


## **Q13. Rolling average of daily revenue**

### Task

Calculate:

* 3-day rolling average of daily revenue

### 💡 Hint

Use:

* `.rolling(window=3)`
* `.mean()`

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.rolling.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.rolling.html)

### 🧠 Explanation

Rolling metrics are used to:

* smooth noisy data
* reveal trends
* support **time-series visualization**

In [ ]:
daily_revenue["rolling_3day_avg"] = (
    daily_revenue["daily_revenue"]
    .rolling(window=3)
    .mean()
)
#daily_revenue

## **Q14. Detect unusually large orders (outliers)**

### Task

Flag orders where:

* `total_amount` is significantly higher than normal
  (use a statistical threshold)

### 💡 Hint

Use:

* `.mean()`
* `.std()`
* boolean conditions

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html)

### 🧠 Explanation

You are learning:

* **basic anomaly detection**
* how math supports intuition
* how analysts decide what deserves investigation

In [30]:
mean_amount = df["total_amount"].mean()
std_amount = df["total_amount"].std()


In [31]:
threshold = mean_amount + 3 * std_amount


In [32]:
outlier_orders = df[df["total_amount"] > threshold]


In [ ]:
#outlier_orders